In [1]:
import numpy as np
%matplotlib notebook
from func import *

In [2]:
SPEED = 0
THETA = 1
T_DROP = 2
T_DETONATE = 3

$$
\begin{aligned}
& x := (v_1, \theta_1, t_{11}, t_{11}') \\
& f(x) := -\lambda\circ\tau(M_1, S_{11}, v_1, \theta_1, t_{11}, t_{11}') \\
& \arg\min_x f(x) = \arg\max \lambda\circ\tau(M_1, S_{11}, v_1, \theta_1, t_{11}, t_{11}')
\end{aligned}
$$

$$
f()
$$

In [3]:
bounds = [(fy_speed_min, fy_speed_max), (0, 2 * np.pi), (0, missile_life[0]), (0, missile_life[0])]

In [4]:
def standardize(params):
    std = np.zeros(len(params))
    for i in range(len(params)):
        std[i] = (params[i] - bounds[i][0]) / (bounds[i][1] - bounds[i][0])
    return std
def inverse(std):
    params = np.zeros(len(std))
    for i in range(len(params)):
        params[i] = bounds[i][0] + std[i] * (bounds[i][1] - bounds[i][0])
    return params

In [5]:
def func(x):
    speed, theta, t_drop, t_detonate = inverse(x)
    set_fy_mission(0, speed, theta)
    interval = get_time_interval(0, 0, t_drop, t_detonate)
    period = get_lebesgue(interval)
    loss = -period
    return loss

In [6]:
initial_param = standardize(np.array([120, 3.141592653589793, 1.5, 3.6]))
initial_loss = func(initial_param)

In [7]:
def get_grad(param: np.ndarray, step: float):
    grad = np.zeros(len(param))
    for i, d in enumerate(np.eye(len(param))):
        dy = func(param + d * step) - func(param - d * step)
        dx = step * 2
        grad[i] = dy / dx
    return grad

## 在Q1的参数的邻域内做网格搜索

In [8]:
search_params = []
search_losses = []

In [9]:
"""for v in np.arange(0.7140, 0.7145, 0.0001):
    for theta in np.arange(0.4895, 0.4975, 0.001):
        param = np.array([v, theta, 0.02238834, 0.05373201])
        loss = loss_func(param)
        print(param, loss)
        search_params.append(param)
        search_losses.append(loss)
search_params_array = np.array(search_params)
search_losses_array = np.array(search_losses)"""

'for v in np.arange(0.7140, 0.7145, 0.0001):\n    for theta in np.arange(0.4895, 0.4975, 0.001):\n        param = np.array([v, theta, 0.02238834, 0.05373201])\n        loss = loss_func(param)\n        print(param, loss)\n        search_params.append(param)\n        search_losses.append(loss)\nsearch_params_array = np.array(search_params)\nsearch_losses_array = np.array(search_losses)'

In [10]:
lr = 1e-3
param = initial_param.copy()
opt_params = [initial_param.copy()]
opt_losses = [initial_loss]

for i in range(100):

    grad = get_grad(param, lr)
    if norm(grad) == 0:
        lr *= 0.5
        print("norm grad = 0", lr)
        continue
    grad /= norm(grad)

    param -= grad * lr
    loss = func(param)

    if opt_losses[-1] + 2e-3 < loss:
        param = opt_params[-1].copy()
        lr *= 0.5
        print("loss up to:", loss, lr)
        continue

    print(i, param, loss)
    opt_params.append(param)
    opt_losses.append(loss)

opt_params_array = np.array(opt_params)
opt_losses_array = np.array(opt_losses)

0 [0.71426754 0.49923649 0.02204294 0.05427737] -1.9400000000000004
1 [0.71425799 0.49860626 0.02163233 0.05493625] -2.460000000000001
2 [0.71423834 0.49809532 0.02119018 0.05567318] -3.000000000000001
3 [0.7142274  0.49810626 0.02068705 0.05653725] -3.42
4 [0.71421622 0.49771519 0.02024011 0.05734173] -3.95
5 [0.71420321 0.49783229 0.01971967 0.05818746] -4.260000000000001
loss up to: -4.19 0.0005
loss up to: -4.23 0.00025
8 [0.71420321 0.49817427 0.01908087 0.05885479] -4.29
loss up to: -4.26 0.000125
10 [0.71420321 0.49810856 0.0189265  0.05883921] -4.300000000000001
loss up to: -4.29 6.25e-05
12 [0.71420321 0.49816446 0.01884265 0.05883921] -4.300000000000001
13 [0.71420321 0.49812027 0.01879845 0.05883921] -4.300000000000001
14 [0.71420321 0.49805777 0.01879845 0.05883921] -4.3100000000000005
loss up to: -4.300000000000001 3.125e-05
loss up to: -4.300000000000001 1.5625e-05
norm grad = 0 7.8125e-06
norm grad = 0 3.90625e-06
norm grad = 0 1.953125e-06
norm grad = 0 9.765625e-07
nor

In [12]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

axis_x = 3
axis_y = 2
ax.scatter(
    search_params_array[:, axis_x],
    search_params_array[:, axis_y],
    search_losses_array[:],
    label="search"
)
"""ax.scatter(
    opt_params_array[:, axis_x],
    opt_params_array[:, axis_y],
    opt_losses_array[:],
    label="opt"
)"""
ax.scatter(
    initial_param[axis_x],
    initial_param[axis_y],
    initial_loss,
    label="initial"
)


plt.legend()
plt.show()

<IPython.core.display.Javascript object>

NameError: name 'search_params_array' is not defined